# Setup

1. Initialize Spaark session connecting to the Postgres DB
2. 

In [ ]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()

from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("MusicBrainz PostgreSQL Connection") \
    .config("spark.jars", "/Users/d.veragillard/edu/semester/WIM-1/big-data-advanced-database/bd-project/postgresql-42.7.1.jar") \
    .getOrCreate()

# Database connection properties
database_url = "jdbc:postgresql://localhost:5432/musicbrainz"
properties = {"user": "musicbrainz", "password": "musicbrainz", "driver": "org.postgresql.Driver"}

# TODO Explore / Clean data

# Reading data from PostgreSQL
df = spark.read.jdbc(url=database_url, table="artist", properties=properties)

# Show the DataFrame
df.show()


In [ ]:
# Example of loading tables from PostgreSQL
def load_table(table_name):
    return spark.read \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://localhost:5432/musicbrainz") \
        .option("dbtable", table_name) \
        .option("user", "musicbrainz") \
        .option("password", "musicbrainz") \
        .load()

# Load necessary tables
artist_df = load_table("artist")
release_df = load_table("release")
recording_df = load_table("recording")
instrument_df = load_table("instrument")
work_df = load_table("work")

# Joining and creating features
# Example: Joining artist and recording tables
# joined_df = recording_df.join(artist_df, recording_df.artist_credit == artist_df.id, "inner")

# Feature Engineering
# Example: Extracting year from date and calculating age of recording
# from pyspark.sql.functions import year, current_date, datediff
# joined_df = joined_df.withColumn("release_year", year(col("release_date")))
# joined_df = joined_df.withColumn("age_of_recording", datediff(current_date(), col("release_date")))

# Show the DataFrame
# joined_df.show()
